# **Preprocesado conjunto Test AIRBNB FORECASTER**

In [1]:
import os

# Cambiar la ruta al nuevo directorio
new_path = r"d:\AI Bootcamp Github\bnb_forecaster_ML\notebooks" 
os.chdir(new_path)

# Confirmar la ruta actual
print("La ruta actual es:", os.getcwd())


La ruta actual es: d:\AI Bootcamp Github\bnb_forecaster_ML\notebooks


In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as mplot
%matplotlib inline 

In [3]:
#cargar datos test y preprocesamiento

df_bnb_test = pd.read_csv("./data/airbnb-listings-cleaned_test.csv",sep=';', decimal='.', encoding='utf-8')
print("bnb_test Shape:", df_bnb_test .shape)
pd.set_option('display.max_columns', None)
df_bnb_test .head(5)

bnb_test Shape: (2956, 49)


,summary,description,host_total_listings_count,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,availability_30,availability_60,availability_90,availability_365,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,cancellation_policy,reviews_per_month
0,el edificio esta junto a un maravilloso parque...,el edificio esta junto a un maravilloso parque...,1.0,villaverde,los rosales,villaverde,madrid,comunidad de madrid,28021,madrid,"madrid, spain",es,spain,40.346320,-3.691440,apartment,private room,1,1.0,1.0,1.0,real bed,"tv,wireless internet,elevator in building,buzz...",NaN,15.0,NaN,NaN,NaN,5.0,1,0,1,1125,7,16,46,46,11,20160913.0,20170403.0,100.0,10.0,10.0,10.0,10.0,9.0,10.0,flexible,1.59
1,es una habitacion con entrada independiente di...,es una habitacion con entrada independiente di...,3.0,palacio,palacio,centro,madrid,comunidad de madrid,28013,madrid,"madrid, spain",es,spain,40.414423,-3.712847,apartment,private room,1,1.0,1.0,1.0,pullout sofa,"tv,internet,wireless internet,air conditioning...",NaN,25.0,101.0,357.0,175.0,20.0,1,0,3,1125,0,5,35,310,17,20160210.0,20170105.0,96.0,10.0,10.0,10.0,10.0,10.0,10.0,moderate,1.20
2,lovely place ideal for 2 couple or 2 friends i...,lovely place ideal for 2 couple or 2 friends i...,1.0,NaN,almagro,chamberi,madrid,comunidad de madrid,28010,madrid,"madrid, spain",es,spain,40.437012,-3.698987,apartment,entire homeapt,2,1.0,1.0,1.0,real bed,"tv,internet,wireless internet,air conditioning...",NaN,45.0,NaN,NaN,150.0,20.0,1,25,1,1125,6,9,12,120,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,strict,NaN
3,alquilo habitacion al lado de casa de campo pa...,alquilo habitacion al lado de casa de campo pa...,1.0,NaN,lucero,latina,madrid,comunidad de madrid,28011,madrid,"madrid, spain",es,spain,40.407932,-3.744533,apartment,private room,1,1.0,1.0,1.0,real bed,"internet,wireless internet,kitchen,free parkin...",NaN,25.0,150.0,NaN,NaN,NaN,1,0,1,1125,28,58,88,363,1,20161204.0,20161204.0,100.0,10.0,10.0,10.0,10.0,10.0,10.0,moderate,0.24
4,"en pleno centro de madrid,coqueta buahardilla ...","en pleno centro de madrid,coqueta buahardilla ...",2.0,NaN,embajadores,centro,madrid,comunidad de madrid,28012,madrid,"madrid, spain",es,spain,40.410107,-3.704869,apartment,entire homeapt,3,1.0,0.0,2.0,real bed,"tv,air conditioning,kitchen,smoking allowed,pe...",NaN,40.0,250.0,500.0,NaN,NaN,1,0,1,1125,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,flexible,NaN


In [4]:
df_bnb_train = pd.read_csv("./data/airbnb-listings-cleaned_train.csv",sep=';', decimal='.', encoding='utf-8')

In [5]:
import numpy as np
import pandas as pd
import joblib

# Cargar las medianas calculadas durante el entrenamiento
group_medians_train = joblib.load("./data/joblib/group_medians_train.joblib")
global_median_train = joblib.load("./data/joblib/global_median_train.joblib")

# Reemplazar 0 por NaN en square_feet en el conjunto de test
df_bnb_test['square_feet'] = df_bnb_test['square_feet'].replace(0, np.nan)

# Imputar valores nulos en square_feet con la mediana AGRUPADA (calculada en TRAIN)
df_bnb_test['square_feet'] = df_bnb_test.apply(
    lambda row: (
        group_medians_train.loc[
            (row['property_type'], row['bedrooms'], row['accommodates'])
        ]
        if pd.isna(row['square_feet']) 
           and (row['property_type'], row['bedrooms'], row['accommodates']) in group_medians_train.index
        else row['square_feet']
    ),
    axis=1
)

# Si aún quedan nulos, imputarlos con la mediana GLOBAL (también del TRAIN)
df_bnb_test['square_feet'] = df_bnb_test['square_feet'].fillna(global_median_train)

# Crear la columna square_meters
df_bnb_test['square_meters'] = df_bnb_test['square_feet'] * 0.092903


In [6]:
# Eliminar square_feet de df_bnb_test para evitar duplicación
df_bnb_test = df_bnb_test.drop(columns=['square_feet'])

In [7]:
import pandas as pd

# Ver cuántos valores nulos hay en test inicialmente
print("Valores nulos en summary (test):", df_bnb_test['summary'].isnull().sum())
print("Valores nulos en description (test):", df_bnb_test['description'].isnull().sum())

# Imputar 'summary' con 'description' y viceversa dentro de la misma fila
df_bnb_test['summary'] = df_bnb_test.apply(
    lambda row: row['description'] if pd.isna(row['summary']) and pd.notna(row['description']) else row['summary'],
    axis=1
)
df_bnb_test['description'] = df_bnb_test.apply(
    lambda row: row['summary'] if pd.isna(row['description']) and pd.notna(row['summary']) else row['description'],
    axis=1
)

# Verificar cuántos valores nulos quedan tras la imputación cruzada
print("Valores nulos después de imputación entre valores de mismo registro (test):")
print("Summary nulos:", df_bnb_test['summary'].isnull().sum())
print("Description nulos:", df_bnb_test['description'].isnull().sum())

# Imputar con valores genéricos si aún quedan nulos
df_bnb_test['summary'] = df_bnb_test['summary'].fillna('No summary available')
df_bnb_test['description'] = df_bnb_test['description'].fillna('No description provided')

# Comprobar nuevamente que queden 0 nulos
print("Valores nulos en 'summary' y 'description' después de la imputación (test):")
print("Summary nulos:", df_bnb_test['summary'].isnull().sum())
print("Description nulos:", df_bnb_test['description'].isnull().sum())


Valores nulos en summary (test): 122
Valores nulos en description (test): 0
Valores nulos después de imputación entre valores de mismo registro (test):
Summary nulos: 0
Description nulos: 0
Valores nulos en 'summary' y 'description' después de la imputación (test):
Summary nulos: 0
Description nulos: 0


In [8]:
import joblib
from sklearn.impute import KNNImputer

# Cargar el imputador ajustado en TRAIN
knn_imputer = joblib.load("./data/joblib/knn_imputer.joblib")

# Sincronizar categorías entre TRAIN y TEST sin usar .cat
categorical_columns = ["property_type", "room_type", "bed_type"]

for col in categorical_columns:
    # Obtener categorías únicas en TRAIN
    train_categories = df_bnb_train[col].dropna().unique()
    
    # Reemplazar categorías desconocidas en TEST por "unknown"
    df_bnb_test[col] = df_bnb_test[col].apply(lambda x: x if x in train_categories else "unknown")
    
    # Asegurarnos de que todos los valores estén en las categorías de TRAIN
    df_bnb_test[col] = df_bnb_test[col].astype(str)

# Convertir categóricas a códigos numéricos para KNN
columns_to_impute = ["bathrooms", "bedrooms", "beds"]
features_for_knn = ["property_type", "square_meters", "accommodates",
                    "room_type", "bed_type"] + columns_to_impute

knn_data_test = df_bnb_test[features_for_knn].copy()

# Mapear valores categóricos a índices numéricos
for col in categorical_columns:
    unique_categories = sorted(df_bnb_train[col].dropna().unique())
    category_map = {cat: idx for idx, cat in enumerate(unique_categories)}
    category_map["unknown"] = len(category_map)  # Añadir "unknown" como categoría adicional
    knn_data_test[col] = knn_data_test[col].map(category_map)

# Imputar con el KNNImputer entrenado (solo transform)
knn_data_test_imputed = knn_imputer.transform(knn_data_test)
knn_data_test_imputed_df = pd.DataFrame(knn_data_test_imputed, columns=knn_data_test.columns)

# Actualizar las columnas en df_bnb_test
df_bnb_test["bathrooms"] = knn_data_test_imputed_df["bathrooms"]
df_bnb_test["bedrooms"]  = knn_data_test_imputed_df["bedrooms"].round().astype(int)
df_bnb_test["beds"]      = knn_data_test_imputed_df["beds"].round().astype(int)

# Confirmar que no hay nulos en las columnas imputadas
print("Valores nulos en TEST después de imputación:")
print(df_bnb_test[["bathrooms", "bedrooms", "beds"]].isnull().sum())


Valores nulos en TEST después de imputación:
bathrooms    0
bedrooms     0
beds         0
dtype: int64


c:\Users\kshad\anaconda3\envs\py310_env\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator KNNImputer from version 1.6.0 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [9]:
import pandas as pd
import joblib

# Cargar las medianas calculadas en train
price_median_train = joblib.load("./data/joblib/price_median_train.joblib")
weekly_price_median_train = joblib.load("./data/joblib/weekly_price_median_train.joblib")
monthly_price_median_train = joblib.load("./data/joblib/monthly_price_median_train.joblib")

# Imputar weekly_price y monthly_price basados en price
df_bnb_test['weekly_price'] = df_bnb_test['weekly_price'].fillna(df_bnb_test['price'] * 7)
df_bnb_test['monthly_price'] = df_bnb_test['monthly_price'].fillna(df_bnb_test['price'] * 30)

# Imputar valores restantes con las medianas de train
df_bnb_test['price'] = df_bnb_test['price'].fillna(price_median_train)
df_bnb_test['weekly_price'] = df_bnb_test['weekly_price'].fillna(weekly_price_median_train)
df_bnb_test['monthly_price'] = df_bnb_test['monthly_price'].fillna(monthly_price_median_train)


In [10]:
import joblib

# Cargar las estadísticas calculadas en train
group_medians_security_deposit = joblib.load("./data/joblib/group_medians_security_deposit.joblib")
group_medians_cleaning_fee = joblib.load("./data/joblib/group_medians_cleaning_fee.joblib")
general_medians = joblib.load("./data/joblib/general_medians.joblib")

# Aplicar imputación en test usando las estadísticas cargadas
df_bnb_test['security_deposit'] = df_bnb_test.apply(
    lambda row: group_medians_security_deposit.get((row['property_type'], row['bedrooms'], row['smart_location']), general_medians['security_deposit'])
    if pd.isnull(row['security_deposit']) else row['security_deposit'], axis=1
)

df_bnb_test['cleaning_fee'] = df_bnb_test.apply(
    lambda row: group_medians_cleaning_fee.get((row['property_type'], row['bedrooms'], row['smart_location']), general_medians['cleaning_fee'])
    if pd.isnull(row['cleaning_fee']) else row['cleaning_fee'], axis=1
)

# Imputar cualquier valor restante con la mediana general
cols_to_impute_with_median = ['square_meters', 'security_deposit', 'cleaning_fee']
for col in cols_to_impute_with_median:
    df_bnb_test[col] = df_bnb_test[col].fillna(general_medians[col])


In [11]:
import joblib

# Cargar las estadísticas calculadas en train
group_medians_review_scores = joblib.load("./data/joblib/group_medians_review_scores.joblib")
global_medians_review_scores = joblib.load("./data/joblib/global_medians_review_scores.joblib")

# Definir columnas a imputar
review_score_cols = [
    'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 
    'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 
    'review_scores_value', 'reviews_per_month'
]

# Imputación en test usando las medianas por grupo calculadas en train
for col in review_score_cols:
    df_bnb_test[col] = df_bnb_test.apply(
        lambda row: group_medians_review_scores[col].get(
            (row['property_type'], row['room_type'], row['number_of_reviews']), 
            global_medians_review_scores[col]
        ) if pd.isnull(row[col]) else row[col], axis=1
    )

# Imputación con la mediana global para cualquier nulo restante
for col in review_score_cols:
    df_bnb_test[col] = df_bnb_test[col].fillna(global_medians_review_scores[col])

# Verificar que no haya nulos en las columnas
print("Nulos restantes en las columnas de puntuación de reseñas y 'reviews_per_month' después de la imputación:")
print(df_bnb_test[review_score_cols].isnull().sum())


Nulos restantes en las columnas de puntuación de reseñas y 'reviews_per_month' después de la imputación:
review_scores_rating           0
review_scores_accuracy         0
review_scores_cleanliness      0
review_scores_checkin          0
review_scores_communication    0
review_scores_location         0
review_scores_value            0
reviews_per_month              0
dtype: int64


In [12]:
import pandas as pd
from datetime import datetime
import joblib

# Cargar la fecha actual y las medianas calculadas en train
today_date = joblib.load("./data/joblib/today_date.joblib")
medians_days_reviews = joblib.load("./data/joblib/medians_days_reviews.joblib")

# Convertir 'first_review' y 'last_review' en formato fecha si no lo están
df_bnb_test['first_review'] = pd.to_datetime(df_bnb_test['first_review'], format='%Y%m%d', errors='coerce')
df_bnb_test['last_review'] = pd.to_datetime(df_bnb_test['last_review'], format='%Y%m%d', errors='coerce')

# Calcular los días desde la fecha actual
df_bnb_test['days_since_first_review'] = df_bnb_test['first_review'].apply(
    lambda x: (today_date - x).days if pd.notna(x) else None
)
df_bnb_test['days_since_last_review'] = df_bnb_test['last_review'].apply(
    lambda x: (today_date - x).days if pd.notna(x) else None
)

# Imputar valores nulos utilizando las medianas calculadas en train
df_bnb_test['days_since_first_review'] = df_bnb_test['days_since_first_review'].fillna(medians_days_reviews['days_since_first_review'])
df_bnb_test['days_since_last_review'] = df_bnb_test['days_since_last_review'].fillna(medians_days_reviews['days_since_last_review'])

# Calcular la duración de la actividad de reseñas
df_bnb_test['review_activity_duration'] = (
    df_bnb_test['days_since_first_review'] - df_bnb_test['days_since_last_review']
)


In [13]:
# Eliminar las columnas originales de fecha
df_bnb_test.drop(columns=['first_review', 'last_review'], inplace=True)

In [14]:
import pandas as pd
import joblib

# Cargar el diccionario de amenities comunes calculado en train
amenities_comunes_dict = joblib.load("./data/joblib/amenities_comunes_dict.joblib")

# Asegurarse de que cada valor de 'amenities_list' sea una lista
df_bnb_test['amenities_list'] = df_bnb_test['amenities'].apply(
    lambda x: x.split(' ') if isinstance(x, str) else []
)

# Imputación de nulos basada en property_type y room_type
df_bnb_test['amenities'] = df_bnb_test.apply(
    lambda row: amenities_comunes_dict.get(
        (row['property_type'], row['room_type']), 
        amenities_comunes_dict["default"]  # Usar "default" si no se encuentra la combinación
    ) if pd.isna(row['amenities']) else row['amenities'], axis=1
)

# Eliminar columna temporal amenities_list si no es necesaria después de la imputación
df_bnb_test.drop(columns=['amenities_list'], inplace=True)

# Verificar si hay nulos en amenities después de la imputación
print("Nulos restantes en la columna 'amenities' después de la imputación:")
print(df_bnb_test['amenities'].isnull().sum())


Nulos restantes en la columna 'amenities' después de la imputación:
0


In [15]:
import pandas as pd
import joblib

# Cargar las estadísticas calculadas en train
median_per_property_type = joblib.load("./data/joblib/median_per_property_type_host_total_listings.joblib")
median_host_total_listings = joblib.load("./data/joblib/median_host_total_listings.joblib")

# Imputar 'host_total_listings_count' utilizando la mediana por 'property_type' del conjunto de entrenamiento
df_bnb_test['host_total_listings_count'] = df_bnb_test.apply(
    lambda row: median_per_property_type.get(row['property_type'], median_host_total_listings) 
    if pd.isna(row['host_total_listings_count']) else row['host_total_listings_count'], axis=1
)

# Imputar cualquier valor restante con la mediana general calculada en train
df_bnb_test['host_total_listings_count'] = df_bnb_test['host_total_listings_count'].fillna(median_host_total_listings)


In [16]:
import pandas as pd
import joblib

# Cargar las estadísticas calculadas en train
country_mode_by_country_code = joblib.load("./data/joblib/country_mode_by_country_code.joblib")
country_mode_general = joblib.load("./data/joblib/country_mode_general.joblib")

# Imputar 'country' basado en 'country_code' utilizando la moda calculada en train
df_bnb_test['country'] = df_bnb_test.apply(
    lambda row: country_mode_by_country_code.get(row['country_code'], country_mode_general)
    if pd.isna(row['country']) else row['country'], axis=1
)

# Imputar cualquier valor restante con la moda general calculada en train
df_bnb_test['country'] = df_bnb_test['country'].fillna(country_mode_general)


In [17]:
import pandas as pd
import joblib

# Cargar las estadísticas calculadas en train
state_mode_by_country = joblib.load("./data/joblib/state_mode_by_country.joblib")
state_mode_general = joblib.load("./data/joblib/state_mode_general.joblib")

# Función para imputar 'state' basado en 'country'
def imputar_state_test(row):
    if pd.isna(row['state']):
        return state_mode_by_country.get(row['country'], state_mode_general)  # Busca en el diccionario o usa la moda general
    return row['state']

# Aplicar la imputación
df_bnb_test['state'] = df_bnb_test.apply(imputar_state_test, axis=1)

# Imputar cualquier valor restante con la moda general calculada en train
df_bnb_test['state'] = df_bnb_test['state'].fillna(state_mode_general)


In [18]:
import pandas as pd
import joblib

# Cargar las estadísticas calculadas en train
market_mode_by_country_state = joblib.load("./data/joblib/market_mode_by_country_state.joblib")
market_mode_general = joblib.load("./data/joblib/market_mode_general.joblib")

# Función para imputar 'market' basado en 'country' y 'state'
def imputar_market_test(row):
    if pd.isna(row['market']):
        return market_mode_by_country_state.get((row['country'], row['state']), market_mode_general)
    return row['market']

# Aplicar la imputación
df_bnb_test['market'] = df_bnb_test.apply(imputar_market_test, axis=1)

# Imputar cualquier valor restante con la moda general calculada en train
df_bnb_test['market'] = df_bnb_test['market'].fillna(market_mode_general)


In [19]:
import pandas as pd
import joblib

# Cargar las estadísticas calculadas en train
city_mode_by_state_market = joblib.load("./data/joblib/city_mode_by_state_market.joblib")
city_mode_general = joblib.load("./data/joblib/city_mode_general.joblib")

# Función para imputar 'city' basado en 'state' y 'market'
def imputar_city_test(row):
    if pd.isna(row['city']):
        return city_mode_by_state_market.get((row['state'], row['market']), city_mode_general)
    return row['city']

# Aplicar la imputación
df_bnb_test['city'] = df_bnb_test.apply(imputar_city_test, axis=1)

# Imputar cualquier valor restante con la moda general calculada en train
df_bnb_test['city'] = df_bnb_test['city'].fillna(city_mode_general)


In [20]:
import joblib
from sklearn.neighbors import KNeighborsClassifier

# Convertir 'zipcode' a cadena para asegurar consistencia
df_bnb_test['zipcode'] = df_bnb_test['zipcode'].astype(str)

# Cargar el modelo entrenado
knn = joblib.load("./data/joblib/knn_zipcode_imputer2.joblib")

# Predecir 'zipcode' solo si hay filas con valores nulos
df_no_zipcode = df_bnb_test[df_bnb_test['zipcode'].isna()]
if not df_no_zipcode.empty:
    # Realizar la predicción y asignarla a las filas correspondientes
    predicted_zipcodes = knn.predict(df_no_zipcode[['latitude', 'longitude']])
    df_bnb_test.loc[df_bnb_test['zipcode'].isna(), 'zipcode'] = predicted_zipcodes

# Verificar si quedan nulos en la columna 'zipcode'
print("Nulos restantes en 'zipcode' después de la imputación:", df_bnb_test['zipcode'].isnull().sum())


Nulos restantes en 'zipcode' después de la imputación: 0


c:\Users\kshad\anaconda3\envs\py310_env\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator KNeighborsClassifier from version 1.6.0 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [21]:
import joblib
from sklearn.neighbors import KNeighborsClassifier
import numpy as np

# Convertir 'neighbourhood_cleansed' a tipo categórico (string) para asegurar consistencia
df_bnb_test['neighbourhood_cleansed'] = df_bnb_test['neighbourhood_cleansed'].astype(str)

# Cargar el modelo entrenado
knn_neighbourhood_cleansed = joblib.load("./data/joblib/knn_neighbourhood_cleansed2.joblib")

# Verificar si hay valores nulos en 'latitude' o 'longitude'
if df_bnb_test[['latitude', 'longitude']].isnull().any().any():
    raise ValueError("Existen valores nulos en 'latitude' o 'longitude' en test. Por favor, impute estos valores primero.")

# Predecir 'neighbourhood_cleansed' para las filas con valores nulos en esa columna
df_no_neighbourhood_cleansed = df_bnb_test[df_bnb_test['neighbourhood_cleansed'].isna()]
if not df_no_neighbourhood_cleansed.empty:
    # Realizar la predicción solo si hay suficientes datos
    try:
        predicted_neighbourhoods_cleansed = knn_neighbourhood_cleansed.predict(df_no_neighbourhood_cleansed[['latitude', 'longitude']])
    except ValueError as e:
        print("No se pudo predecir debido a datos insuficientes:", e)
        predicted_neighbourhoods_cleansed = np.full(len(df_no_neighbourhood_cleansed), "unknown")  # Valor por defecto

    # Asignar los valores predichos o el fallback
    df_bnb_test.loc[df_bnb_test['neighbourhood_cleansed'].isna(), 'neighbourhood_cleansed'] = predicted_neighbourhoods_cleansed

# Verificar si quedan nulos en 'neighbourhood_cleansed'
print("Nulos restantes en 'neighbourhood_cleansed' después de la imputación:", df_bnb_test['neighbourhood_cleansed'].isnull().sum())


Nulos restantes en 'neighbourhood_cleansed' después de la imputación: 0


c:\Users\kshad\anaconda3\envs\py310_env\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator KNeighborsClassifier from version 1.6.0 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [22]:
from sklearn.neighbors import KNeighborsClassifier
import joblib

# Cargar el modelo KNN entrenado
knn = joblib.load("./data/joblib/knn_neighbourhood_group_cleansed2.joblib")

# Aplicar imputación jerárquica en test
def imputar_neighbourhood_group_test(row):
    if pd.isna(row['neighbourhood_group_cleansed']):
        # Coincidencia exacta con todos los campos
        subset = df_bnb_train[
            (df_bnb_train['zipcode'] == row['zipcode']) &
            (df_bnb_train['latitude'] == row['latitude']) &
            (df_bnb_train['longitude'] == row['longitude']) &
            (df_bnb_train['city'] == row['city'])
        ]

        if subset.empty:
            subset = df_bnb_train[
                (df_bnb_train['zipcode'] == row['zipcode']) &
                (df_bnb_train['city'] == row['city'])
            ]
        
        if subset.empty:
            subset = df_bnb_train[df_bnb_train['zipcode'] == row['zipcode']]
        
        if not subset.empty and not subset['neighbourhood_group_cleansed'].mode().empty:
            return subset['neighbourhood_group_cleansed'].mode()[0]
    return row['neighbourhood_group_cleansed']

# Aplicar la imputación jerárquica en test
df_bnb_test['neighbourhood_group_cleansed'] = df_bnb_test.apply(imputar_neighbourhood_group_test, axis=1)

# Aplicar KNN para registros restantes
df_no_neighbourhood_group_test = df_bnb_test[df_bnb_test['neighbourhood_group_cleansed'].isna()]
if not df_no_neighbourhood_group_test.empty:
    df_bnb_test.loc[df_bnb_test['neighbourhood_group_cleansed'].isna(), 'neighbourhood_group_cleansed'] = knn.predict(df_no_neighbourhood_group_test[['latitude', 'longitude']])

# Verificar nulos restantes
print("Nulos restantes en 'neighbourhood_group_cleansed' después de la imputación:", df_bnb_test['neighbourhood_group_cleansed'].isnull().sum())


c:\Users\kshad\anaconda3\envs\py310_env\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator KNeighborsClassifier from version 1.6.0 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Nulos restantes en 'neighbourhood_group_cleansed' después de la imputación: 0


In [23]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
import joblib
import numpy as np

# Cargar los LabelEncoders y el modelo KNN entrenado
label_encoders = joblib.load("./data/joblib/label_encoders_neighbourhood.joblib")
knn = joblib.load("./data/joblib/knn_neighbourhood2.joblib")

# Copiar el DataFrame original
temp_df = df_bnb_test.copy()

# Transformar columnas categóricas a valores numéricos con verificación previa
for col in ['neighbourhood_cleansed', 'neighbourhood_group_cleansed']:
    le = label_encoders[col]
    temp_df[col] = temp_df[col].apply(lambda x: le.transform([x])[0] if x in le.classes_ else -1)

# Verificar si se encontraron valores nuevos y asignar un valor por defecto
for col in ['neighbourhood_cleansed', 'neighbourhood_group_cleansed']:
    le = label_encoders[col]
    if -1 in temp_df[col].values:
        print(f"Se encontraron valores desconocidos en {col}, asignando 'unknown'.")
        temp_df[col] = temp_df[col].replace(-1, len(le.classes_))  # Asignar un índice nuevo para 'unknown'

# Predecir 'neighbourhood' para registros sin valor
df_no_neighbourhood = temp_df[temp_df['neighbourhood'].isna()]
if not df_no_neighbourhood.empty:
    predicted_neighbourhoods = knn.predict(df_no_neighbourhood[['latitude', 'longitude', 'neighbourhood_cleansed', 'neighbourhood_group_cleansed']])
    df_bnb_test.loc[df_bnb_test['neighbourhood'].isna(), 'neighbourhood'] = predicted_neighbourhoods

# Verificar si quedan nulos en 'neighbourhood'
print("Nulos restantes en 'neighbourhood' después de la imputación:", df_bnb_test['neighbourhood'].isnull().sum())


c:\Users\kshad\anaconda3\envs\py310_env\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.2.2 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\kshad\anaconda3\envs\py310_env\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator KNeighborsClassifier from version 1.6.0 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Se encontraron valores desconocidos en neighbourhood_cleansed, asignando 'unknown'.
Se encontraron valores desconocidos en neighbourhood_group_cleansed, asignando 'unknown'.
Nulos restantes en 'neighbourhood' después de la imputación: 0


In [24]:
import pandas as pd
import joblib

# Cargar las categorías de amenidades actualizadas
categories = joblib.load("./data/joblib/amenity_categories_updated.joblib")

# Asignar pesos a cada categoría
high_weight = 3
standard_weight = 2
basic_weight = 1
default_weight = 0  # Peso para amenidades desconocidas

# Diccionario para reemplazar variantes o errores en las amenidades
amenity_replacements = {
    'pack n playtravel crib': 'pack ’n play/travel crib',
    'self checkin': 'self check-in',
    'childrens dinnerware': 'children’s dinnerware',
    'childrens books and toys': 'children’s books and toys',
    'wheelchair': 'wheelchair accessible',
    'free parking on street': 'free parking on premises',
    'washer  dryer': 'washer / dryer',
    '24hour checkin': '24-hour check-in',
    'buzzerwireless intercom': 'buzzer/wireless intercom',
    'translation': None,  # Eliminar irrelevantes
    'missing enhostingamenity50 on enhostingamenity49': None,
}

# Función para limpiar las amenidades
def clean_amenities_test(amenities):
    if pd.isna(amenities):
        return None
    amenities_list = amenities.lower().split(',')
    cleaned_list = [amenity_replacements.get(amenity.strip(), amenity.strip()) for amenity in amenities_list]
    return ','.join(filter(None, cleaned_list))  # Eliminar amenidades irrelevantes

# Limpiar las categorías
df_bnb_test['amenities'] = df_bnb_test['amenities'].apply(clean_amenities_test)

# Función para calcular el puntaje de amenidades en test y registrar nuevas amenidades
def calculate_amenity_score_test(amenities, new_amenities=set()):
    if pd.isna(amenities):
        return 0  # Si no hay amenidades, el puntaje será 0
    
    # Dividir las amenidades en una lista de amenidades individuales
    amenities_list = amenities.lower().split(',')
    
    # Contadores para cada categoría
    high_count = sum(amenity.strip() in categories["high"] for amenity in amenities_list)
    standard_count = sum(amenity.strip() in categories["standard"] for amenity in amenities_list)
    basic_count = sum(amenity.strip() in categories["basic"] for amenity in amenities_list)
    
    # Identificar amenidades nuevas
    for amenity in amenities_list:
        if amenity.strip() not in categories["high"] + categories["standard"] + categories["basic"]:
            categories["basic"].append(amenity.strip())  # Asignar automáticamente a la categoría básica
            new_amenities.add(amenity.strip())
            # Serializar categorías actualizadas inmediatamente
            joblib.dump(categories, "./data/joblib/amenity_categories_updated.joblib")
    
    # Calcular el puntaje total
    return (
        (high_count * high_weight) +
        (standard_count * standard_weight) +
        (basic_count * basic_weight)
    )

# Crear un conjunto para rastrear nuevas amenidades
new_amenities_test = set()

# Aplicar la función para calcular el puntaje de amenidades
df_bnb_test['amenity_score'] = df_bnb_test['amenities'].apply(
    lambda x: calculate_amenity_score_test(x, new_amenities=new_amenities_test)
)

# Mostrar amenidades nuevas encontradas en test
print("Amenidades nuevas asignadas a la categoría básica en test:", new_amenities_test)


Amenidades nuevas asignadas a la categoría básica en test: set()


In [25]:
# Eliminar la columna original de 'amenities' si ya no es necesaria
df_bnb_test = df_bnb_test.drop(columns=['amenities'])

In [26]:
import pandas as pd

# Definir las mismas palabras clave relevantes utilizadas en train
relevant_keywords = [
    'gran via', 'living room', 'puerta sol', 'plaza mayor', 'centro madrid',
    'fully equipped', 'double bed', 'reina sofia', 'lugares interes', 'madrid', 
    'apartment', 'room', 'metro', 'plaza', 'sol', 'bed', 'kitchen', 'centro', 
    'located', 'apartamento', 'habitacion', 'barrio', 'wifi', 'cocina', 'gran',
    'minutes', 'puerta', 'minutos', 'area', 'living', 'casa', 'city', 'cama', 
    'bathroom', 'two', 'piso', 'double', 'bedroom', 'center', 'via', 'zona', 
    'min', 'bano', 'quiet', 'walk', 'tv', 'equipped', 'center madrid', 
    'alojamiento bueno', 'sofa bed', 'walking distance', 'apartment located',
    'heart madrid', 'washing machine', 'pleno centro', 'va encantar', 
    'equipped kitchen', 'plaza espana'
]

# Función para calcular el puntaje de contenido basado en las palabras clave
def calculate_content_score(text):
    if pd.isna(text):
        return 0  # Si el texto es nulo, el puntaje será 0
    text = text.lower()
    score = sum(1 for phrase in relevant_keywords if phrase in text)
    return score

# Aplicar la función a las columnas `summary` y `description`
df_bnb_test['summary_content_score'] = df_bnb_test['summary'].apply(calculate_content_score)
df_bnb_test['description_content_score'] = df_bnb_test['description'].apply(calculate_content_score)

# Verificar los resultados
print("Test: Puntajes de contenido calculados")
print(df_bnb_test[['summary_content_score', 'description_content_score']].head())

# Guardar los puntajes relevantes como archivo CSV o pickle para futuras referencias
df_bnb_test[['summary_content_score', 'description_content_score']].to_csv(
    './data/test_content_scores.csv', index=False
)


Test: Puntajes de contenido calculados
   summary_content_score  description_content_score
0                      9                         11
1                      7                         14
2                     13                         16
3                     11                         15
4                      3                          3


In [27]:
import pandas as pd
import joblib

# Cargar el mapeo y los rangos de precio desde train
property_type_group_mapping = joblib.load("./data/joblib/property_type_mapping.joblib")
price_ranges = joblib.load("./data/joblib/price_ranges.joblib")

# Función para manejar categorías nuevas
def assign_new_category(row):
    if pd.isna(row['property_type_grouped']):  # Si es una categoría nueva
        # Calcular el precio promedio de la categoría en test
        subset = df_bnb_test[df_bnb_test['property_type'] == row['property_type']]
        if not subset.empty:
            avg_price = subset['price'].mean()
            # Asignar al grupo basado en los rangos de precio promedio
            for group, avg_group_price in price_ranges.items():
                if avg_price <= avg_group_price:  # Comparar con el promedio del grupo
                    return group
            return 'high_price_accommodation'  # Si supera todos los rangos
        else:
            return 'low_price_accommodation'  # Si no hay suficiente información
    return row['property_type_grouped']

# Asignar agrupaciones a property_type_grouped en test
df_bnb_test['property_type_grouped'] = df_bnb_test['property_type'].map(property_type_group_mapping)

# Aplicar lógica para manejar categorías nuevas
df_bnb_test['property_type_grouped'] = df_bnb_test.apply(assign_new_category, axis=1)

# Verificar resultados
print("Valores nulos en property_type_grouped (test):", df_bnb_test['property_type_grouped'].isna().sum())

# Guardar el DataFrame procesado
df_bnb_test.to_csv("./data/test_property_type_processed.csv", index=False)
print("\nProcesamiento de test completado y guardado.")


Valores nulos en property_type_grouped (test): 0

Procesamiento de test completado y guardado.


In [28]:
import pandas as pd
import joblib

# Cargar el mapeo de agrupaciones y los pesos desde train
property_type_group_mapping = joblib.load("./data/joblib/property_type_mapping.joblib")
property_type_group_weights = joblib.load("./data/joblib/property_type_weights.joblib")
price_ranges = joblib.load("./data/joblib/price_ranges.joblib")

# Función para manejar categorías nuevas en test
def assign_new_category_and_weight(row):
    if pd.isna(row['property_type_grouped']):  # Si es una categoría nueva
        subset = df_bnb_test[df_bnb_test['property_type'] == row['property_type']]
        if not subset.empty:
            avg_price = subset['price'].mean()
            for group, avg_group_price in price_ranges.items():
                if avg_price <= avg_group_price:
                    return property_type_group_weights.get(group, 1)  # Asignar peso correspondiente
            return property_type_group_weights.get('high_price_accommodation', 4)  # Peso máximo
        else:
            return 1  # Peso mínimo por defecto
    else:
        # Retornar peso del grupo existente
        return property_type_group_weights.get(row['property_type_grouped'], 1)

# Asignar agrupaciones a property_type_grouped
df_bnb_test['property_type_grouped'] = df_bnb_test['property_type'].map(property_type_group_mapping)

# Calcular los pesos para las categorías nuevas y existentes
df_bnb_test['property_type_weighted'] = df_bnb_test.apply(assign_new_category_and_weight, axis=1)

# Verificar valores nulos después del mapeo
print("Valores nulos en property_type_weighted (test):", df_bnb_test['property_type_weighted'].isna().sum())

# Guardar el DataFrame procesado
df_bnb_test.to_csv("./data/test_property_type_weighted.csv", index=False)
print("\nProcesamiento de test completado y guardado.")


Valores nulos en property_type_weighted (test): 0

Procesamiento de test completado y guardado.


In [29]:
# Eliminar columnas redundantes
df_bnb_test = df_bnb_test.drop(columns=['property_type', 'property_type_grouped'])

In [30]:
import pandas as pd
import joblib

# Cargar los mapeos guardados en train
room_type_mean = joblib.load("./data/joblib/room_type_mean.joblib")
bed_type_mean = joblib.load("./data/joblib/bed_type_mean.joblib")

# Mapear los valores al dataset de test
df_bnb_test['room_type_encoded'] = df_bnb_test['room_type'].map(room_type_mean)
df_bnb_test['bed_type_encoded'] = df_bnb_test['bed_type'].map(bed_type_mean)

# Tratar valores nulos generados por categorías nuevas o ausentes
# Usamos el promedio general de 'price' en test como imputación
df_bnb_test['room_type_encoded'].fillna(df_bnb_test['price'].mean(), inplace=True)
df_bnb_test['bed_type_encoded'].fillna(df_bnb_test['price'].mean(), inplace=True)

# Verificar el resultado final
print("Valores nulos en room_type_encoded (test):", df_bnb_test['room_type_encoded'].isna().sum())
print("Valores nulos en bed_type_encoded (test):", df_bnb_test['bed_type_encoded'].isna().sum())

# Guardar el DataFrame procesado
df_bnb_test.to_csv("./data/test_room_bed_encoded.csv", index=False)
print("\nArchivo final de test guardado.")


Valores nulos en room_type_encoded (test): 0
Valores nulos en bed_type_encoded (test): 0

Archivo final de test guardado.


C:\Users\kshad\AppData\Local\Temp\ipykernel_460\141549071.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_bnb_test['room_type_encoded'].fillna(df_bnb_test['price'].mean(), inplace=True)
C:\Users\kshad\AppData\Local\Temp\ipykernel_460\141549071.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values alwa

In [31]:
# Eliminar las columnas originales después de la codificación
df_bnb_test = df_bnb_test.drop(columns=['room_type', 'bed_type'])

In [32]:
import pandas as pd
import joblib

# Cargar el mapeo de cancellation_policy desde train
cancellation_p_map = joblib.load("./data/joblib/cancellation_policy_map.joblib")

# Convertir cancellation_policy a numérica en test
df_bnb_test['cancellation_policy_numeric'] = df_bnb_test['cancellation_policy'].map(cancellation_p_map)

# Tratar valores nulos generados por categorías nuevas o ausentes
df_bnb_test['cancellation_policy_numeric'].fillna(0, inplace=True)  # Asignar 0 como categoría desconocida

# Verificar valores nulos después de la conversión
print("Valores nulos en cancellation_policy_numeric (test):", df_bnb_test['cancellation_policy_numeric'].isna().sum())

# Guardar el DataFrame procesado
df_bnb_test.to_csv("./data/test_cancellation_policy_encoded.csv", index=False)
print("\nArchivo final de test guardado.")


Valores nulos en cancellation_policy_numeric (test): 0

Archivo final de test guardado.


C:\Users\kshad\AppData\Local\Temp\ipykernel_460\3134258152.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_bnb_test['cancellation_policy_numeric'].fillna(0, inplace=True)  # Asignar 0 como categoría desconocida


In [33]:
# Eliminar la columna original 'cancellation_policy' para evitar redundancia
df_bnb_test = df_bnb_test.drop(columns=['cancellation_policy'])

In [34]:
import joblib
import pandas as pd

# Cargar el mapa de medias existente
mean_map_neighbourhoods = joblib.load("./data/joblib/mean_map_neighbourhoods.joblib")

# Aplicar Target Encoding a `neighbourhood_cleansed` (si está disponible en test)
if 'neighbourhood_cleansed' in df_bnb_test.columns:
    df_bnb_test['neighbourhood_cleansed_encoded'] = df_bnb_test['neighbourhood_cleansed'].map(mean_map_neighbourhoods['neighbourhood_cleansed'])
    # Imputar nulos con la media global del precio en TRAIN
    df_bnb_test['neighbourhood_cleansed_encoded'].fillna(df_bnb_train['price'].mean(), inplace=True)

# Aplicar Target Encoding a `neighbourhood` (si está disponible en test)
if 'neighbourhood' in df_bnb_test.columns:
    df_bnb_test['neighbourhood_encoded'] = df_bnb_test['neighbourhood'].map(mean_map_neighbourhoods['neighbourhood'])
    # Imputar nulos con la media global del precio en TRAIN
    df_bnb_test['neighbourhood_encoded'].fillna(df_bnb_train['price'].mean(), inplace=True)

# Verificar si quedan nulos después de la imputación
print("Verificación de nulos después de la imputación:")
print(df_bnb_test[['neighbourhood_cleansed_encoded', 'neighbourhood_encoded']].isnull().sum())

# Mostrar las primeras filas para confirmar resultados
print(df_bnb_test[['neighbourhood_cleansed', 'neighbourhood_cleansed_encoded', 'neighbourhood', 'neighbourhood_encoded']].head())




Verificación de nulos después de la imputación:
neighbourhood_cleansed_encoded    0
neighbourhood_encoded             0
dtype: int64
  neighbourhood_cleansed  neighbourhood_cleansed_encoded neighbourhood  \
0            los rosales                       23.000000    villaverde   
1                palacio                       77.303855       palacio   
2                almagro                       80.226891       almagro   
3                 lucero                       30.638298        aluche   
4            embajadores                       60.703779     la latina   

   neighbourhood_encoded  
0              72.041096  
1              81.323913  
2             106.273885  
3              42.192547  
4              67.881178  


C:\Users\kshad\AppData\Local\Temp\ipykernel_460\3932062765.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_bnb_test['neighbourhood_cleansed_encoded'].fillna(df_bnb_train['price'].mean(), inplace=True)
C:\Users\kshad\AppData\Local\Temp\ipykernel_460\3932062765.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are set

In [35]:
# Eliminar las columnas categóricas originales después del encoding
df_bnb_test.drop(columns=['neighbourhood', 'neighbourhood_cleansed', 'neighbourhood_group_cleansed'], inplace=True)

In [36]:
# Eliminar la columna 'smart_location' del DataFrame de entrenamiento
df_bnb_test = df_bnb_test.drop(columns=['smart_location'])

In [37]:
import pandas as pd
import joblib

# Cargar los mapeos de Target Encoding guardados en train
mean_map = joblib.load("./data/joblib/mean_map_location_features.joblib")
target_column = 'price'  # Columna objetivo para predecir

# Aplicar Target Encoding a 'city', 'state', 'zipcode', y 'market' en test
for column in ['city', 'state', 'zipcode', 'market']:
    df_bnb_test[f'{column}_encoded'] = df_bnb_test[column].map(mean_map[column])

    # Manejar valores nulos generados por categorías nuevas o ausentes
    df_bnb_test[f'{column}_encoded'].fillna(df_bnb_test[target_column].mean(), inplace=True)

# Verificar valores nulos después del mapeo
for column in ['city', 'state', 'zipcode', 'market']:
    print(f"Valores nulos en {column}_encoded (test):", df_bnb_test[f'{column}_encoded'].isnull().sum())



Valores nulos en city_encoded (test): 0
Valores nulos en state_encoded (test): 0
Valores nulos en zipcode_encoded (test): 0
Valores nulos en market_encoded (test): 0


C:\Users\kshad\AppData\Local\Temp\ipykernel_460\2513528557.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_bnb_test[f'{column}_encoded'].fillna(df_bnb_test[target_column].mean(), inplace=True)
C:\Users\kshad\AppData\Local\Temp\ipykernel_460\2513528557.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting val

In [38]:
import pandas as pd
import joblib

# Cargar el mapeo de Target Encoding guardado en train
mean_map = joblib.load("./data/joblib/mean_map_country.joblib")
target_column = 'price'  # Columna objetivo para predecir

# Aplicar Target Encoding a 'country'
df_bnb_test['country_encoded'] = df_bnb_test['country'].map(mean_map['country'])

# Manejar valores nulos generados por categorías nuevas o ausentes
df_bnb_test['country_encoded'].fillna(df_bnb_test[target_column].mean(), inplace=True)

# Eliminar la columna 'country_code' para evitar duplicación de información
df_bnb_test.drop(columns=['country_code'], inplace=True)

# Confirmar que la columna 'country_code' fue eliminada
print("\nColumnas después de aplicar Target Encoding y eliminar 'country_code' (test):")
print(df_bnb_test.columns)

# Guardar el DataFrame procesado
df_bnb_test.to_csv("./data/test_country_encoded.csv", index=False)
print("\nArchivo final de test guardado.")



Columnas después de aplicar Target Encoding y eliminar 'country_code' (test):
Index(['summary', 'description', 'host_total_listings_count', 'city', 'state',
       'zipcode', 'market', 'country', 'latitude', 'longitude', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'price', 'weekly_price',
       'monthly_price', 'security_deposit', 'cleaning_fee', 'guests_included',
       'extra_people', 'minimum_nights', 'maximum_nights', 'availability_30',
       'availability_60', 'availability_90', 'availability_365',
       'number_of_reviews', 'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'reviews_per_month', 'square_meters',
       'days_since_first_review', 'days_since_last_review',
       'review_activity_duration', 'amenity_score', 'summary_content_score',
       'description_content_score', 'property_type_weighted',
     

C:\Users\kshad\AppData\Local\Temp\ipykernel_460\1743700788.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_bnb_test['country_encoded'].fillna(df_bnb_test[target_column].mean(), inplace=True)


In [39]:
# Eliminar columnas originales para evitar redundancia
df_bnb_test.drop(columns=['city', 'state', 'zipcode', 'market'], inplace=True)

In [40]:
import numpy as np

# Crear una nueva columna con el logaritmo natural de 'price'
df_bnb_test['price_log'] = np.log(df_bnb_test['price'])

# Eliminar la columna 'price'
df_bnb_test.drop(columns=['price'], inplace=True)

# Verificar las primeras filas
print(df_bnb_test.head())


                                             summary  \
0  el edificio esta junto a un maravilloso parque...   
1  es una habitacion con entrada independiente di...   
2  lovely place ideal for 2 couple or 2 friends i...   
3  alquilo habitacion al lado de casa de campo pa...   
4  en pleno centro de madrid,coqueta buahardilla ...   

                                         description  \
0  el edificio esta junto a un maravilloso parque...   
1  es una habitacion con entrada independiente di...   
2  lovely place ideal for 2 couple or 2 friends i...   
3  alquilo habitacion al lado de casa de campo pa...   
4  en pleno centro de madrid,coqueta buahardilla ...   

   host_total_listings_count country   latitude  longitude  accommodates  \
0                        1.0   spain  40.346320  -3.691440             1   
1                        3.0   spain  40.414423  -3.712847             1   
2                        1.0   spain  40.437012  -3.698987             2   
3                     

In [41]:
# Calcular la media simple para las columnas de calificaciones en test
df_bnb_test['review_overall_score'] = df_bnb_test[
    ['review_scores_rating', 'review_scores_cleanliness', 'review_scores_location',
     'review_scores_communication', 'review_scores_value', 'number_of_reviews', 'review_scores_accuracy', 'review_scores_checkin',
       'reviews_per_month']
].mean(axis=1)

# Eliminar las columnas originales después de calcular la media en test
df_bnb_test.drop(columns=[
    'review_scores_rating', 'review_scores_cleanliness', 'review_scores_location',
    'review_scores_communication', 'review_scores_value', 'number_of_reviews', 'review_scores_accuracy', 'review_scores_checkin',
       'reviews_per_month'
], inplace=True)

# Verificar el resultado
print("Columnas después de procesar las reseñas en test:")
print(df_bnb_test.columns)


Columnas después de procesar las reseñas en test:
Index(['summary', 'description', 'host_total_listings_count', 'country',
       'latitude', 'longitude', 'accommodates', 'bathrooms', 'bedrooms',
       'beds', 'weekly_price', 'monthly_price', 'security_deposit',
       'cleaning_fee', 'guests_included', 'extra_people', 'minimum_nights',
       'maximum_nights', 'availability_30', 'availability_60',
       'availability_90', 'availability_365', 'square_meters',
       'days_since_first_review', 'days_since_last_review',
       'review_activity_duration', 'amenity_score', 'summary_content_score',
       'description_content_score', 'property_type_weighted',
       'room_type_encoded', 'bed_type_encoded', 'cancellation_policy_numeric',
       'neighbourhood_cleansed_encoded', 'neighbourhood_encoded',
       'city_encoded', 'state_encoded', 'zipcode_encoded', 'market_encoded',
       'country_encoded', 'price_log', 'review_overall_score'],
      dtype='object')


In [42]:
df_bnb_test.drop(columns=['availability_30',
       'availability_60', 'availability_90', 'minimum_nights', 'maximum_nights', 'days_since_first_review', 'days_since_last_review'], inplace=True)

In [43]:
# Eliminar la columna redundante 'monthly_price y weekly_price'
df_bnb_test.drop(columns=['monthly_price', 'weekly_price'], inplace=True)

In [44]:
#eliminare la columna extra_people pues tiene outliers muy grandes y numeros que no concuerdan con la cantidad de habitaciones, camas y personas en el inmueble, por ejemplo 500 personas?
df_bnb_test.drop(columns=[
    'extra_people'
], inplace=True)

In [45]:
# Eliminar la columna neighbourhood_cleansed_encoded para evitar redundancias
df_bnb_test.drop(columns=['neighbourhood_cleansed_encoded'], inplace=True)

# Confirmar que la columna fue eliminada
print("Columnas después de eliminar'neighbourhood_cleansed_encoded':")
print(df_bnb_test.columns)

Columnas después de eliminar'neighbourhood_cleansed_encoded':
Index(['summary', 'description', 'host_total_listings_count', 'country',
       'latitude', 'longitude', 'accommodates', 'bathrooms', 'bedrooms',
       'beds', 'security_deposit', 'cleaning_fee', 'guests_included',
       'availability_365', 'square_meters', 'review_activity_duration',
       'amenity_score', 'summary_content_score', 'description_content_score',
       'property_type_weighted', 'room_type_encoded', 'bed_type_encoded',
       'cancellation_policy_numeric', 'neighbourhood_encoded', 'city_encoded',
       'state_encoded', 'zipcode_encoded', 'market_encoded', 'country_encoded',
       'price_log', 'review_overall_score'],
      dtype='object')


In [46]:
# Eliminar las columnas seleccionadas
columns_to_drop = ['state_encoded', 'market_encoded', 'country_encoded', 'latitude', 'longitude']
df_bnb_test.drop(columns=columns_to_drop, inplace=True)

In [47]:
# Eliminar la columna 'host_total_listings_count'
df_bnb_test.drop(columns=['host_total_listings_count'], inplace=True)

In [48]:
# Drop original text columns after extraction
df_bnb_test = df_bnb_test.drop(columns=['summary', 'description'])

In [49]:
# Drop original text columns after extraction
df_bnb_test = df_bnb_test.drop(columns=['country'])

In [55]:

# Ruta de los archivos

outlier_limits_path = r"D:\AI Bootcamp Github\bnb_forecaster_ML\notebooks\data\joblib\outlier_limits.pkl"

# Cargar los límites de outliers
outlier_limits = joblib.load(outlier_limits_path)

# Filtrar los datos del conjunto de prueba
for column, (lower, upper) in outlier_limits.items():
    if column in df_bnb_test.columns:
        df_bnb_test = df_bnb_test[(df_bnb_test[column] >= lower) & (df_bnb_test[column] <= upper)]




In [57]:
# Cargar las columnas categóricas
categorical_columns_path = r"D:\AI Bootcamp Github\bnb_forecaster_ML\notebooks\data\joblib\categorical_columns.pkl"
categorical_columns = joblib.load(categorical_columns_path)

# Cargar escalador guardado
scaler_path = r"D:\AI Bootcamp Github\bnb_forecaster_ML\notebooks\data\joblib\scaler.pkl"
scaler = joblib.load(scaler_path)

# Escalar columnas categóricas en test
df_bnb_test[categorical_columns] = scaler.transform(df_bnb_test[categorical_columns])



In [58]:

# Crear nuevas características basadas en las reglas de entrenamiento
df_bnb_test['price_per_guest'] = df_bnb_test['price_log'] / (df_bnb_test['accommodates'] + 1)  # Precio por huésped
df_bnb_test['total_beds'] = df_bnb_test['beds'] * df_bnb_test['bedrooms']  # Total de camas



In [59]:
# Eliminar la misma columna eliminada en train
df_bnb_test.drop(columns=['neighbourhood_encoded'], inplace=True)

In [60]:
# Cargar el escalador guardado
scaler = joblib.load(r"D:\AI Bootcamp Github\bnb_forecaster_ML\notebooks\models\minmax_pricescaler.pkl")

# Actualizar directamente price_log
df_bnb_test['price_log'] = scaler.transform(df_bnb_test[['price_log']])




In [61]:
# guardar df_bnb_train_finalpreprocess
df_bnb_test.to_csv(r"D:\AI Bootcamp Github\bnb_forecaster_ML\notebooks\data\df_bnb_test_finalpreprocess.csv", index=False)

In [62]:
# Ruta completa del archivo
file_path = r"D:\AI Bootcamp Github\bnb_forecaster_ML\notebooks\data\df_bnb_test_finalpreprocess.csv"

# Cargar el DataFrame desde el archivo CSV
df_bnb_test = pd.read_csv(file_path)

# Verificar las primeras filas
print(df_bnb_test.head())

   accommodates  bathrooms  bedrooms  beds  security_deposit  cleaning_fee  \
0             1        1.0         1     1             150.0           5.0   
1             1        1.0         1     1             175.0          20.0   
2             2        1.0         1     1             150.0          20.0   
3             1        1.0         1     1             150.0          20.0   
4             3        1.0         0     2             150.0          25.0   

   guests_included  availability_365  square_meters  review_activity_duration  \
0                1                46      19.974145                     202.0   
1                1               310      19.974145                     330.0   
2                1               120      24.990907                     359.0   
3                1               363      19.974145                       0.0   
4                1                 0      44.965052                     359.0   

   amenity_score  summary_content_score  des

In [64]:
for column in sorted(df_bnb_test.columns):
    print(column)


accommodates
amenity_score
availability_365
bathrooms
bed_type_encoded
bedrooms
beds
cancellation_policy_numeric
city_encoded
cleaning_fee
description_content_score
guests_included
price_log
price_per_guest
property_type_weighted
review_activity_duration
review_overall_score
room_type_encoded
security_deposit
square_meters
summary_content_score
total_beds
zipcode_encoded


In [65]:
df_bnb_test.shape

(2722, 23)